In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [50]:
!pip install transformers
!pip install spacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 52.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
! pip install tqdm

#Filtering the name entities that were incorrected labeled with the spaCy library

In [7]:
import pandas as pd
from transformers import pipeline
from tqdm.notebook import tqdm


df = pd.read_csv("/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/entities_frequency.csv")


ner_pipeline = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", tokenizer="mrm8488/bert-spanish-cased-finetuned-ner", grouped_entities=True)


def is_correctly_tagged(entity):
    try:
        results = ner_pipeline(entity)
        for r in results:
            if r['entity_group'] in ['LOC', 'ORG']:
                return True
    except:
        pass
    return False


tqdm.pandas()
df['correctly_tagged'] = df['entity'].progress_apply(is_correctly_tagged)
filtered_df = df[df['correctly_tagged'] == True].drop(columns=['correctly_tagged'])


filtered_df.to_csv("/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/filtered_entities.csv", index=False)
print(filtered_df)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


  0%|          | 0/16145 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


             entity type  frequency
1            Madrid  LOC       1911
3            España  LOC       1055
4           Dolores  LOC        898
9             Santa  LOC        486
10          Sevilla  LOC        389
...             ...  ...        ...
16124   Casa-Malaña  LOC          1
16129      Amazonas  LOC          1
16134  Puerto-Arcas  LOC          1
16137     Montceaux  LOC          1
16140         Arcas  LOC          1

[3691 rows x 3 columns]


In [5]:
filtered_df.shape

(3691, 3)

#Sentiment Analysis of a selected group of exotic locations

In [7]:


import os
import glob
import random
import xml.etree.ElementTree as ET
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

sentiment_analyzer = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")

route = "/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/spa"
files = glob.glob(os.path.join(route, "*_annotated.xml"))

# Key words
key_words = [
    "Francia", "Inglaterra", "París", "Roma", "Italia", "Flandes", "Londres",
    "América", "Portugal", "Nápoles", "Maya", "Fátima", "Alemania", "Rajatul-laj",
    "Occidente", "Austria", "África", "Túnez", "Birmingham", "Habana", "Lisboa",
    "Cuba", "Bruselas", "Méjico", "Biarritz", "Indias", "Asia", "Egipto",
    "Constantinopla", "Holanda", "Damasco", "Filipinas", "India", "Bretaña",
    "Rusia", "Grecia", "Escocia"
]

def extract_tokens(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}
    tokens = []
    for w in root.findall('.//tei:w', ns):
        text = w.text
        if text:
            tokens.append({
                "text": text,
                "pos": w.attrib.get("pos", ""),
                "lemma": w.attrib.get("lemma", ""),
                "ner": w.attrib.get("ner", "")
            })
    return tokens

# Obtain the year of publication in order to compare periods
def get_year(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}
    date_node = root.find('.//tei:sourceDesc//tei:bibl[@type="firstEdition"]/tei:date', ns)
    if date_node is not None and date_node.text and date_node.text.strip().isdigit():
        return int(date_node.text.strip())
    return None

# Classify the years in 2 decade periods
def classify_year(year):
    if year is None:
        return "unknown"
    if 1840 <= year <= 1859:
        return "1840-1859"
    elif 1860 <= year <= 1879:
        return "1860-1879"
    elif 1880 <= year <= 1899:
        return "1880-1899"
    elif 1900 <= year <= 1920:
        return "1900-1920"
    return "Out of range"

# Get windows of text
def windows(tokens, key_word, file_, period, window=100):
    windows = []
    for i, token in enumerate(tokens):
        if token["text"] == key_word:
            start = max(i - window, 0)
            end = min(i + window + 1, len(tokens))
            context = " ".join([tok["text"] for tok in tokens[start:end]])
            windows.append({
                "kew_word": key_word,
                "text_window": context,
                "file": os.path.basename(file_),
                "period": period
            })
    return windows

# Get all the windows that contain the key_word
all_windows = []

for file_ in tqdm(files, desc="Processing all files"):
    year = get_year(file_)
    period = classify_year(year)
    tokens = extract_tokens(file_)

    for word in key_words:
        new = windows(tokens, word, file_, period)
        all_windows.extend(new)

# Sentiment Analysis in all the windows that have one key_word on the text
for window in tqdm(all_windows, desc="Analyzing sentiments"):
    try:
        result = sentiment_analyzer(window["text_window"][:512])[0]
        window["sentiment"] = result["label"]
        window["confidence_score"] = result["score"]
    except Exception as e:
        window["sentiment"] = "error"
        window["confidence_score"] = 0.0


df = pd.DataFrame(all_windows)

# Select all windows that mention a key word (country) with the best confidence score
df_top30 = (
    df.sort_values(by=["key_word", "confidence_score"], ascending=[True, False])
      .groupby("key_word")
      .head(30)
      .reset_index(drop=True)
)

df_top30.to_csv("/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/data/top30_sentiment.csv", index=False)

print("Df saved.")

Device set to use cuda:0
Analyzing sentiments: 100%|██████████| 4992/4992 [00:32<00:00, 154.23it/s]


Df saved.


In [62]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/data/top30_sentiments.csv")

In [64]:
df['period'].value_counts()

,count
period,
1900-1920,402
1880-1899,226
1840-1859,217
1860-1879,175
unknown,67


#Analysis of the results

In [65]:
conteo = df.groupby('key_word')['sentiment'].value_counts().unstack(fill_value=0)
print(conteo)

sentiment       Negative  Neutral  Positive  Very Negative  Very Positive
key_word                                                                 
Alemania               0        5         4             14              7
América                0        3         1             13             13
Asia                   0        8         6              9              7
Austria                0        3         0             10             17
Biarritz               0       10         4             10              6
Birmingham             1        5         1             17              6
Bretaña                1        5         5             12              7
Bruselas               0        4         3             18              5
Constantinopla         0        4         2             18              6
Cuba                   0        4         3             16              7
Damasco                0        1         4              2              1
Egipto                 0       12     

# Collapsing the sentiment labels

## Selecting only the countries by interest (Asia, America, Egypt, Philippines and Cuba)

In [66]:
import re

df['file'] = df['file'].astype(str)

# Modifying the incorrect labeled files
periods = {
    'SPA3029_Blanco_Cercado_annotated.xml': '1880-1899',
    'SPA2021_RafaelDelCastillo_LosCaballerosDelAmor_annotated.xml': '1860-1879',
    'SPA3002_LeopoldoAlas_LaRegenta_annotated.xml': '1900-1920',
    'SPA1022_GomezDeAvellaneda_DosMujeres_annotated.xml': '1840-1859'
}


for file, period in periods.items():
    df.loc[df['file'] == file, 'period'] = period

# Collapsing the labels into 2 and droping the Neutral instances
reduced_sentiment = {
    'Very Negative': 'Negative',
    'Very Positive': 'Positive',
    'Neutral': None
}

df['sentiment'] = df['sentiment'].map(reduced_sentiment)
df = df.dropna(subset=['sentiment'])

In [67]:
df.shape

(775, 7)

In [68]:
# Selecting only some countries to analyse
countries = ['Cuba', 'Asia', 'América', 'Egipto', 'Filipinas']
regex = '|'.join(map(re.escape, countries))
df = df[df['key_word'].str.contains(regex, case=False, na=False)]


In [69]:
df.shape

(103, 7)

In [70]:
df['period'].value_counts()

,count
period,
1880-1899,38
1860-1879,32
1900-1920,21
1840-1859,12


In [71]:
df.groupby('key_word')['sentiment'].value_counts().unstack(fill_value=0)

sentiment,Negative,Positive
key_word,,
América,13,13
Asia,9,7
Cuba,16,7
Egipto,6,11
Filipinas,12,9


In [72]:
df.groupby(['key_word'])['period'].value_counts()

key_word   period   
América    1860-1879    10
           1880-1899     9
           1900-1920     6
           1840-1859     1
Asia       1860-1879     7
           1900-1920     4
           1880-1899     3
           1840-1859     2
Cuba       1880-1899    11
           1860-1879     5
           1900-1920     5
           1840-1859     2
Egipto     1840-1859     6
           1880-1899     4
           1900-1920     4
           1860-1879     3
Filipinas  1880-1899    11
           1860-1879     7
           1900-1920     2
           1840-1859     1
Name: count, dtype: int64

# Pos tagging all windows to get cleaner wordClouds

In [77]:
import os
import spacy
from collections import defaultdict


nlp = spacy.load("es_core_news_sm")


os.makedirs("/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/sentiments", exist_ok=True)

# Dictionary to save the words by group
words_by_group = defaultdict(list)


for _, row in df.iterrows():
    text = row['text_window']
    sentiment = row['sentiment']
    key_word = row['key_word']
    period = row['period']

    doc = nlp(text)

    # POS tag filtering
    filtered_words = [
        token.lemma_.lower() for token in doc
        if token.pos_ in ['NOUN', 'VERB', 'ADJ', 'ADV'] and token.pos_ != 'PROPN'
        and not token.is_stop and token.is_alpha
    ]

    #Saving the files
    file_path = f"{key_word}__{sentiment}_{period}"
    words_by_group[file_path].extend(filtered_words)


for file_path, words in words_by_group.items():
    clean_file_path = re.sub(r'[^\w\-_]', '_', file_path)
    name_file = f"/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/sentiments/{clean_file_path}.txt"

    with open(name_file, 'w', encoding='utf-8') as f:
        f.write(" ".join(words))


# Cleaning with tf-idf

In [79]:
import os

input_dir = '/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/sentiments'


words_by_file = {}

# Leer todos los archivos .txt
for filename in os.listdir(input_dir):
    if filename.endswith('.txt'):
        filepath = os.path.join(input_dir, filename)
        with open(filepath, 'r', encoding='utf-8') as f:
            words = f.read().split()
            words_by_file[filename] = set(words)

# Obtaining the common words for all of the files
if words_by_file:
    common_words = set.intersection(*words_by_file.values())
else:
    common_words = set()

# Filtering the common words across all files
for filename, words in words_by_file.items():
    filtered_words = words - common_words
    filepath = os.path.join(input_dir, filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(" ".join(filtered_words))

# WordClouds

In [80]:
import os
from wordcloud import WordCloud
import matplotlib.pyplot as plt


output_dir = '/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/sentiments/wordCloud'
os.makedirs(output_dir, exist_ok=True)


input_dir = '/content/drive/MyDrive/Colab.Notebooks/Digital Humanities/Project/sentiments'

# Get all .txt file of sentiments
for filename in os.listdir(input_dir):
    if filename.endswith('.txt'):
        filepath = os.path.join(input_dir, filename)


        with open(filepath, 'r', encoding='utf-8') as file:
            text = file.read()

        # WordCloud Creatin
        wc = WordCloud(
            width=800, height=400,
            background_color='white',
            colormap='viridis'
        ).generate(text)

        # Save Image
        output_path = os.path.join(output_dir, filename.replace('.txt', '.png'))
        wc.to_file(output_path)
